In [1]:
import numpy as np
import pandas as pd
import os.path
import scipy
import cv2
import sys
from lxml import etree
from utils import *
from laneDetection import LaneProcessor
from caffeutil import *
from xmlutil import *
%matplotlib inline

In [2]:
def extractBinarizedImg(imgout,rawimg):
    rawimg = rawimg[:,:,::-1]
    imgBinary = (imgout>100).astype(np.float)

    for i in range(3):
        rawimg[:,:,i] = rawimg[:,:,i]*imgBinary

    return rawimg


def generateAllFrame(sample_dir, xml_INFO, net, BEV_coord, initParams=None, predLinFit=None):
    l_image =  os.listdir("%s" % (sample_dir))
    l_image = sorted(l_image)
    sample = filter(lambda x: len(x)>0, sample_dir.split("/"))[-1]
    outName = "./%s-Result.xml" % (sample)

    coord_3d = BEV_coord['coord_3d']
    coord_6m = BEV_coord['coord_600cm']

    M_RTK = parseInfo(xml_INFO)
    src,jac = cv2.projectPoints(coord_3d, M_RTK['R'],  M_RTK['T'], M_RTK['K'], M_RTK['D'] )
    src = src[src[:,0,:].argsort(axis=0)[:,0],0,:]
    src_6m = cv2.projectPoints(coord_6m, M_RTK['R'],  M_RTK['T'], M_RTK['K'], M_RTK['D'] )[0][0][0]

    dst = np.array([[[100,1000], [100,0], [1100,0], [1100,1000]]]).astype(np.float32)
    M    = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    mm = np.dot(M, np.array(list(src_6m) + [1.]))
    xy_trans = (mm / mm[2])[0:2]

    refPos = xy_trans
    l_name = []
    l_nameRaw = []
    l_t_x = []
    l_t_y = []
    l_errors = []

    seedParams = []
    l_ret =  []
    #print(l_image)
    for i in range(len(l_image)):
    #for i in range(1):
        inName  = "%s/%s" % (sample_dir, l_image[i])
        caffeOutImg,rawImg = roadLaneFromSegNet(inName, net)
        rawImg2 = rawImg.copy()
        caffeOutImg_color = extractBinarizedImg(caffeOutImg,rawImg2)
    
    
        binarizedImg = caffeOutImg
        binarizedImg_t = cv2.warpPerspective(binarizedImg, M, (rawImg2.shape[1],rawImg2.shape[0]), cv2.WARP_INVERSE_MAP)
    
        #rawImg3 = rawImg.copy()
        binarizedImg2 = caffeOutImg_color
        binarizedImg2_t = cv2.warpPerspective(binarizedImg2, M, (rawImg2.shape[1],rawImg2.shape[0]), cv2.WARP_INVERSE_MAP)
        try:
            lp = LaneProcessor(binarizedImg_t, binarizedImg2_t, Minv)
            l_ret.append(lp.full_process())
        except:
            l_ret.append([])
        
    print(outName)
    writeXML(l_ret, outName)
#return binarizedImg_t,binarizedImg2_t,rawImg, Minv

In [3]:
model = "../SegNet-Tutorial/Example_Models/segnet_model_driving_webdemo.prototxt"
weights = "../SegNet-Tutorial/Example_Models/segnet_weights_driving_webdemo.caffemodel"

width   = 500
x_start = 200
x_end   = 5000
coord_3d = np.float32([
        [x_start,-width,0], [x_start, width,0],
        [x_end,  -width,0], [x_end,   width,0]
]).reshape(-1,3)
coord_6m = np.array([600., 0., 0.]).reshape(-1,3)
BEV_coords = {
        "coord_3d":coord_3d,
        "coord_600cm" : coord_6m
}
initParams = [
        np.linspace(0.001, 0.02, 10),
        np.array(list(np.linspace(1.5, 50, 10)) + list(np.linspace(-50, 1.5, 10))),
        np.arange( 23, 28, 2),
        np.arange(-15, 15, 2)
]
predLinFit = np.array([ 6.66317499,  1.35541496])
net = loadNet(model, weights)
    

In [4]:
dirs ="/home/xufq/Downloads/hubq/datasets/TSD-Lane/TSD-Lane-00056"#"/home/xufq/Downloads/hubq/datasets/TSD-LKSM/TSD-LKSM-00159/"
info ="/home/xufq/Downloads/hubq/SegNet-Tutorial/xml/TSD-Lane-Info/TSD-Lane-00056-Info.xml" #"/home/xufq/Downloads/hubq/SegNet-Tutorial/xml/TSD-LKSM-Info/TSD-LKSM-00159-Info.xml"
generateAllFrame(dirs,info, net, BEV_coords)

['TSD-Lane-00056-00000.png', 'TSD-Lane-00056-00001.png', 'TSD-Lane-00056-00002.png', 'TSD-Lane-00056-00003.png', 'TSD-Lane-00056-00004.png', 'TSD-Lane-00056-00005.png', 'TSD-Lane-00056-00006.png', 'TSD-Lane-00056-00007.png', 'TSD-Lane-00056-00008.png', 'TSD-Lane-00056-00009.png', 'TSD-Lane-00056-00010.png', 'TSD-Lane-00056-00011.png', 'TSD-Lane-00056-00012.png', 'TSD-Lane-00056-00013.png', 'TSD-Lane-00056-00014.png', 'TSD-Lane-00056-00015.png', 'TSD-Lane-00056-00016.png', 'TSD-Lane-00056-00017.png', 'TSD-Lane-00056-00018.png', 'TSD-Lane-00056-00019.png', 'TSD-Lane-00056-00020.png', 'TSD-Lane-00056-00021.png', 'TSD-Lane-00056-00022.png', 'TSD-Lane-00056-00023.png', 'TSD-Lane-00056-00024.png', 'TSD-Lane-00056-00025.png', 'TSD-Lane-00056-00026.png', 'TSD-Lane-00056-00027.png', 'TSD-Lane-00056-00028.png', 'TSD-Lane-00056-00029.png', 'TSD-Lane-00056-00030.png', 'TSD-Lane-00056-00031.png', 'TSD-Lane-00056-00032.png', 'TSD-Lane-00056-00033.png', 'TSD-Lane-00056-00034.png', 'TSD-Lane-00056-000

utils.py:168: RuntimeWarning: overflow encountered in ubyte_scalars
  if point_rgb[0] + point_rgb[1] < 256. / 255.:


([49673.0], [6117.0])
([59831.0], [5828.0])
([58801.0], [6661.0])
([50501.0], [6136.0])
([75356.0], [6526.0])
([66467.0], [8151.0])
([64800.0], [7123.0])
([60544.0], [6749.0])
([54300.0], [5877.0])
([64648.0], [7760.0])
([63249.0], [8270.0])
([63707.0], [8195.0])
([65324.0], [6732.0])
([64165.0], [8514.0])
([63573.0], [6907.0])
([63847.0], [6751.0])
([67944.0], [5689.0])
([62828.0], [5542.0])
([72400.0], [6191.0])
([69123.0], [6172.0])
([68369.0], [5561.0])
([60220.0], [5579.0])
([53663.0], [5019.0])
([51484.0], [5482.0])
([48800.0], [4745.0])
([45906.0], [5529.0])
([33649.0], [7324.0])
([37538.0], [11856.0])
([43317.0], [6984.0])
([43226.0], [12590.0])
([44997.0], [10227.0])
([52398.0], [8414.0])
([42145.0], [8821.0])
([53385.0], [13048.0])
([53438.0], [7225.0])
([48792.0], [11613.0])
([47720.0], [11400.0])
([44700.0], [14205.0])
([43085.0], [8776.0])
([41341.0], [10087.0])
([42423.0], [11775.0])
([44663.0], [12650.0])
([49249.0], [12747.0])
([48085.0], [13390.0])
([51675.0], [9026.0]

In [ ]:

# color = mpimg.imread(TEST_IMAGE_COLOR)

# warped = mpimg.imread(TEST_IMAGE_NAME)
#current_time = time.time.()
    #for i in range(1):
warped = i1
color = i2

lp = LaneProcessor(warped, color, Minv)
print(lp.full_process())
    #lp.find_curve_function()
    #lp.process_from_raw_coeff_to_result()
    #print(lp.rotated)
    #print(lp.get_coeffs())
#print(time.time() - current_time, 'HAH')
#lp.draw_indicators()
print(lp.rotated)